# eEDB-011-2024-3

## Atividade 3 – Ingestão e ETL com linguagem de programação (Python + Spark)

- Utilizar linguagem de programação Python para ingestão e tratamento de dados. Todo o processo deve ser realizado via Spark.
    - Pacotes adicionais podem ser utilizados
    - Tratamento de dados não deve ser realizado via SQL
- Realizar a ingestão de todas as base de dados em um banco de dados relacional open source. Pode ser utilizado qualquer banco de dado sendo algumas sugestões:
    -  MySQL
    -  Postgre
    - ClickHouse
- Gerar uma tabela final com os dados tratados e unidos.
    - O tratamento de dados deve ser realizado através da linguagem de programação Python + Spark
- Adicionar as seguintes camadas de processamento, dentro do próprio banco de dados ou em disco local. A Camada Delivery deve
obrigatoriamente ter estar também no formato de uma tabela final dentro do banco de dados relacional:
    - RAW – formato dos dados livre
    - Trusted – formato de dados em Parquet ou ORC or AVRO (indicado Parquet)
    - Delivery– formato de dados em Parquet ou ORC or AVRO (indicado Parquet)

- **Grupo 02**:
    - Aline Bini
    - Ana Lívia Franco
    - Ana Priss
    - João Squinelato
    - Marcelo Pena
    - Thais Siqueira

- [Github](https://github.com/Squinelato/eEDB-011-2024-3 "eEDB-011-2024-3")

```Ingestão De Dados | Agosto 2024```

## To Do

- raw 
- trusted
- delivery

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, FloatType, IntegerType

import os

In [2]:
spark = SparkSession.builder.getOrCreate()

---
## **Raw**

### **Banks file**

In [8]:
banks_csv_path = '../Fonte de Dados/Bancos/EnquadramentoInicia_v2.tsv'
rwzd_bank = spark.read.csv(banks_csv_path, sep='\t', encoding='utf8', header=True)
rwzd_bank.show(100, truncate=False)

+--------+--------+--------------------------------------------------------------------------------------------------------------------------+
|Segmento|CNPJ    |Nome                                                                                                                      |
+--------+--------+--------------------------------------------------------------------------------------------------------------------------+
|S1      |0       |BANCO DO BRASIL - PRUDENCIAL                                                                                              |
|S1      |60746948|BRADESCO - PRUDENCIAL                                                                                                     |
|S1      |30306294|BTG PACTUAL - PRUDENCIAL                                                                                                  |
|S1      |360305  |CAIXA ECONOMICA FEDERAL - PRUDENCIAL                                                                                      |

Analisando o esquema dos dados

In [9]:
rwzd_bank.printSchema()

root
 |-- Segmento: string (nullable = true)
 |-- CNPJ: string (nullable = true)
 |-- Nome: string (nullable = true)



Contando a quantidade de linhas

In [10]:
rwzd_bank.count()

1474

Salvando dados na camada _raw_ no formato parquet

In [ ]:
raw_bank_path = './raw/bank/'
rwzd_bank.write.parquet(raw_bank_path, mode="append")

### **Employees file**

Localizando todos os arquivos contendo dados de empregados

In [12]:
employee_dir = '../Fonte de Dados/Empregados/'
employee_files = os.listdir(employee_dir)
employee_paths = list(map(lambda file: os.path.join(employee_dir, file), employee_files))
employee_paths

['../Fonte de Dados/Empregados/glassdoor_consolidado_join_match_less_v2.csv',
 '../Fonte de Dados/Empregados/glassdoor_consolidado_join_match_v2.csv']

Lendo todos os arquivos de empregados como um único conjunto de dados

In [13]:
rwzd_employee = spark.read.csv(employee_paths, sep='|', encoding='utf8', header=True)
rwzd_employee.show()

+--------------------+-------------+-------------+--------------+--------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+-----+-----------------+----------------------+-----------------+--------------+------------------------+-------------------------+---------------------------------+----------------------------------+--------+--------------------+-------------+
|       employer_name|reviews_count|culture_count|salaries_count|benefits_count|    employer-website|employer-headquarters|employer-founded|   employer-industry|    employer-revenue|                 url|Geral|Cultura e valores|Diversidade e inclusão|Qualidade de vida|Alta liderança|Remuneração e benefícios|Oportunidades de carreira|Recomendam para outras pessoas(%)|Perspectiva positiva da empresa(%)|Segmento|                Nome|match_percent|
+--------------------+-------------+-------------+--------------+--------------+--------------------+---

Removendo duplicatas com base no nome e segmento do banco

In [14]:
rwzd_employee = rwzd_employee.dropDuplicates(['Nome', 'Segmento'])

Analisando o esquema dos dados

In [15]:
rwzd_employee.printSchema()

root
 |-- employer_name: string (nullable = true)
 |-- reviews_count: string (nullable = true)
 |-- culture_count: string (nullable = true)
 |-- salaries_count: string (nullable = true)
 |-- benefits_count: string (nullable = true)
 |-- employer-website: string (nullable = true)
 |-- employer-headquarters: string (nullable = true)
 |-- employer-founded: string (nullable = true)
 |-- employer-industry: string (nullable = true)
 |-- employer-revenue: string (nullable = true)
 |-- url: string (nullable = true)
 |-- Geral: string (nullable = true)
 |-- Cultura e valores: string (nullable = true)
 |-- Diversidade e inclusão: string (nullable = true)
 |-- Qualidade de vida: string (nullable = true)
 |-- Alta liderança: string (nullable = true)
 |-- Remuneração e benefícios: string (nullable = true)
 |-- Oportunidades de carreira: string (nullable = true)
 |-- Recomendam para outras pessoas(%): string (nullable = true)
 |-- Perspectiva positiva da empresa(%): string (nullable = true)
 |-- Seg

Contando a quantidade de linhas

In [16]:
rwzd_employee.count()

37

Salvando dados na camada _raw_ no formato parquet

In [ ]:
raw_employee_path = './raw/employee/'
rwzd_employee.write.parquet(raw_employee_path, mode="append")

### **Claims**

Localizando todos os arquivos contendo dados de reclamações

In [17]:
claim_dir = '../Fonte de Dados/Reclamações/'
claim_files = os.listdir(claim_dir)
claim_paths = list(map(lambda file: os.path.join(claim_dir, file), claim_files))
claim_paths

['../Fonte de Dados/Reclamações/2021_tri_01.csv',
 '../Fonte de Dados/Reclamações/2021_tri_02.csv',
 '../Fonte de Dados/Reclamações/2021_tri_03.csv',
 '../Fonte de Dados/Reclamações/2021_tri_04.csv',
 '../Fonte de Dados/Reclamações/2022_tri_01.csv',
 '../Fonte de Dados/Reclamações/2022_tri_02_nao_ha_dados.csv',
 '../Fonte de Dados/Reclamações/2022_tri_03.csv',
 '../Fonte de Dados/Reclamações/2022_tri_04.csv']

In [18]:
rwzd_claim = spark.read.csv(claim_paths, sep=';', encoding='latin1', header=True)
rwzd_claim.show()

+----+---------+--------------------+----------------+--------+----------------------+------+-----------------------------------------------+--------------------------------------------+---------------------------------------+-------------------------------+----------------------------------------+----------------------------+----------------------------+----+
| Ano|Trimestre|           Categoria|            Tipo| CNPJ IF|Instituição financeira|Índice|Quantidade de reclamações reguladas procedentes|Quantidade de reclamações reguladas - outras|Quantidade de reclamações não reguladas|Quantidade total de reclamações|Quantidade total de clientes  CCS e SCR|Quantidade de clientes  CCS|Quantidade de clientes  SCR|_c14|
+----+---------+--------------------+----------------+--------+----------------------+------+-----------------------------------------------+--------------------------------------------+---------------------------------------+-------------------------------+----------------

Analisando o esquema dos dados

In [19]:
rwzd_claim.printSchema()

root
 |-- Ano: string (nullable = true)
 |-- Trimestre: string (nullable = true)
 |-- Categoria: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- CNPJ IF: string (nullable = true)
 |-- Instituição financeira: string (nullable = true)
 |-- Índice: string (nullable = true)
 |-- Quantidade de reclamações reguladas procedentes: string (nullable = true)
 |-- Quantidade de reclamações reguladas - outras: string (nullable = true)
 |-- Quantidade de reclamações não reguladas: string (nullable = true)
 |-- Quantidade total de reclamações: string (nullable = true)
 |-- Quantidade total de clientes  CCS e SCR: string (nullable = true)
 |-- Quantidade de clientes  CCS: string (nullable = true)
 |-- Quantidade de clientes  SCR: string (nullable = true)
 |-- _c14: string (nullable = true)



Removendo coluna desnecessária

In [20]:
rwzd_claim = rwzd_claim.drop('_c14')

Contando a quantidade de linhas

In [21]:
rwzd_claim.count()

918

Salvando dados na camada _raw_ no formato parquet

In [ ]:
raw_claim_path = './raw/claim/'
rwzd_claim.write.parquet(raw_claim_path, mode="append")

---
## **Trusted**

### **Banks**

In [30]:
trzd_bank = rwzd_bank

Aplicando algumas transformações com o intuito de melhorar a qualidade dos dados:

1 - Renomeando colunas do _dataframe_ para inglês e no formato _snake case_

In [39]:
trzd_bank = trzd_bank.withColumnsRenamed({
    'Segmento': 'segment',
    'CNPJ': 'cnpj',
    'Nome': 'financial_institution_name'
})

2 - Para que os dados da coluna _cnpj_ estivessem de acordo com seu padrão, os valores incompletos receberam numerais zeros à esquerda até completar 8 dígitos

In [41]:
def zero_fill_string(string_value: str) -> str:
    return 